In [1]:
from py2neo import Graph
from bs4 import BeautifulSoup
import requests

In [2]:
graph = Graph(password="kshen3778")

In [3]:
graph.run("MATCH (n:Category) RETURN n LIMIT 25").data()

[{'n': (_5214855:Category:Page {id: 690070, isNew: false, isRedirect: false, title: 'Futurama'})},
 {'n': (_5214856:Category:Page {id: 690451, isNew: false, isRedirect: false, title: 'World_War_II'})},
 {'n': (_5214857:Category:Page {id: 690571, isNew: false, isRedirect: false, title: 'Programming_languages'})},
 {'n': (_5214858:Category:Page {id: 690578, isNew: false, isRedirect: false, title: 'Professional_wrestling'})},
 {'n': (_5214859:Category:Page {id: 690637, isNew: false, isRedirect: false, title: 'Algebra'})},
 {'n': (_5214860:Category:Page {id: 690649, isNew: false, isRedirect: false, title: 'Anime'})},
 {'n': (_5214861:Category:Page {id: 690672, isNew: false, isRedirect: false, title: 'Abstract_algebra'})},
 {'n': (_5214862:Category:Page {id: 690747, isNew: false, isRedirect: false, title: 'Mathematics'})},
 {'n': (_5214863:Category:Page {id: 690777, isNew: false, isRedirect: false, title: 'Linear_algebra'})},
 {'n': (_5214864:Category:Page {id: 690803, isNew: false, isRedir

In [20]:
graph.run("MATCH (n:Category { title: 'Cold_War' }) RETURN n").data()

[{'n': (_5214918:Category:Page {id: 691614, isNew: false, isRedirect: false, title: 'Cold_War'})}]

In [55]:
#get all sub cat
cats = graph.run("MATCH (n:Category{title:'Constraint_logic_programming'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()

In [56]:
len(cats)

2

In [57]:
cats

[{'p': (_6124108:Category:Page {id: 22968506, isNew: false, isRedirect: false, title: 'Constraint_programming'})},
 {'p': (_5594005:Category:Page {id: 1188828, isNew: false, isRedirect: false, title: 'Logic_programming'})}]

In [27]:
getBadCategories("Artificial_intelligence")

['Areas_of_computer_science',
 'Cognitive_science',
 'Computational_neuroscience',
 'Cybernetics',
 'Emerging_technologies',
 'Formal_sciences',
 'Futurology',
 'Intelligence_by_type',
 'Personhood',
 'Technology_in_society',
 'Unsolved_problems_in_computer_science']

In [63]:
#check if a specific sub category references the select category an two way relationship (which we don't want)
subs = graph.run("MATCH (n:Category{title:'Emerging_technologies'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()

In [64]:
subs

[]

In [20]:
#verify if category actually exists on online Wikipedia
def categoryExist(name):
    res = requests.get("https://en.wikipedia.org/wiki/Category:" + name)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", {"id": "mw-normal-catlinks"})
    if(div):
        return True
    else:
        return False

#Use beautifulsoup to scrape the bottom categories of a parent category (these we don't want)
def getBadCategories(parent):
    res = requests.get("https://en.wikipedia.org/wiki/Category:" + parent)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", {"id": "mw-normal-catlinks"})
    if(div):
        to_return = []
        for item in div.findAll("li"):
            to_return.append("_".join(item.get_text().split(" ")))
        return to_return
    else:
        #parent does not exist
        return []

#get all immediate Wikipedia subcategories of main minus two way connections
def getAllSubCat(parent):
    all_cats = graph.run("MATCH (n:Category{title:'" + parent + "'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()
    bad_cats = getBadCategories(parent)
    filtered = []
    for category in all_cats:
        cat_title = category['p']["title"]
        if (not (cat_title in bad_cats)):
            filtered.append(cat_title)
    #search all these categories to see if any of them match the other way (if so, we remove)
#     filtered = []
#     for category in all_cats:
#         cat_title = category['p']["title"]
#         subs = graph.run("MATCH (n:Category{title:'" + cat_title + "'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()
#         found = False
#         for category_2 in subs:
#             sub_title = category_2['p']["title"]
#             if(sub_title == parent):
#                 found = True
#         if (not found):
#             filtered.append(cat_title)
    
    return filtered

In [77]:
getAllSubCat("Machine_learning")
#getBadCategories("Constraint_logic_programming")

['Artificial_neural_networks',
 'Applied_machine_learning',
 'Unsupervised_learning',
 'Structured_prediction',
 'Loss_functions',
 'Support_vector_machines',
 'Latent_variable_models',
 'Cluster_analysis',
 'Signal_processing_conferences',
 'Artificial_intelligence_conferences',
 'Data_mining_and_machine_learning_software',
 'Machine_learning_algorithms',
 'Bayesian_networks',
 'Machine_learning_portal',
 'Statistical_natural_language_processing',
 'Markov_models',
 'Evolutionary_algorithms',
 'Genetic_programming',
 'Datasets_in_machine_learning',
 'Dimension_reduction',
 'Machine_learning_researchers',
 'Supervised_learning',
 'Semisupervised_learning',
 'Computational_learning_theory',
 'Kernel_methods_for_machine_learning',
 'Learning_in_computer_vision',
 'Inductive_logic_programming',
 'Log-linear_models',
 'Classification_algorithms',
 'Machine_learning_task',
 'Ensemble_learning']

In [79]:
graph.run("MATCH path = (start:Category{title:'Machine_learning'})-[*]-(end:Category{title:'Court_titles'}) RETURN path LIMIT 3").data()

KeyboardInterrupt: 

In [21]:
all_nodes = []
all_relations = [] #child parent
def buildGraph(root, path):
    path = path + "/" + root
    print("Current Path: " + path)
    if(not categoryExist(root)):
        print("Category DNE")
        return
    #add the node
    all_nodes.append(root)
    
    #get all subcategories
    sub_cat = getAllSubCat(root)
    if(len(sub_cat) == 0): #base case
        return
    
    #add all the relationships
    for item in sub_cat:
        all_relations.append(item + " " + root)
   
    #recurse on all sub nodes
    for item in sub_cat:
        buildGraph(item, path)


In [22]:
buildGraph("Machine_learning", "")

Current Path: /Machine_learning
Current Path: /Machine_learning/Artificial_neural_networks
Current Path: /Machine_learning/Artificial_neural_networks/Neural_network_software
Current Path: /Machine_learning/Artificial_neural_networks/Deep_learning
Current Path: /Machine_learning/Applied_machine_learning
Current Path: /Machine_learning/Applied_machine_learning/AlphaGo
Current Path: /Machine_learning/Unsupervised_learning
Current Path: /Machine_learning/Structured_prediction
Current Path: /Machine_learning/Structured_prediction/Graphical_models
Current Path: /Machine_learning/Structured_prediction/Graphical_models/Bayesian_networks
Current Path: /Machine_learning/Structured_prediction/Graphical_models/Causal_inference
Current Path: /Machine_learning/Structured_prediction/Graphical_models/Markov_networks
Current Path: /Machine_learning/Loss_functions
Current Path: /Machine_learning/Support_vector_machines
Current Path: /Machine_learning/Latent_variable_models
Current Path: /Machine_learnin

In [28]:
all_nodes

['Machine_learning',
 'Artificial_neural_networks',
 'Neural_network_software',
 'Deep_learning',
 'Applied_machine_learning',
 'AlphaGo',
 'Unsupervised_learning',
 'Structured_prediction',
 'Graphical_models',
 'Bayesian_networks',
 'Causal_inference',
 'Markov_networks',
 'Loss_functions',
 'Support_vector_machines',
 'Latent_variable_models',
 'Structural_equation_models',
 'Factor_analysis',
 'Cluster_analysis',
 'Clustering_criteria',
 'Cluster_analysis_algorithms',
 'Signal_processing_conferences',
 'Artificial_intelligence_conferences',
 'Data_mining_and_machine_learning_software',
 'Social_network_analysis_software',
 'Machine_learning_algorithms',
 'Genetic_algorithms',
 'Artificial_immune_systems',
 'Gene_expression_programming',
 'Bayesian_networks',
 'Statistical_natural_language_processing',
 'Language_modeling',
 'Markov_models',
 'Hidden_Markov_models',
 'Markov_networks',
 'Evolutionary_algorithms',
 'Genetic_programming',
 'Genetic_algorithms',
 'Artificial_immune_sys

In [27]:
all_relations

['Artificial_neural_networks Machine_learning',
 'Applied_machine_learning Machine_learning',
 'Unsupervised_learning Machine_learning',
 'Structured_prediction Machine_learning',
 'Loss_functions Machine_learning',
 'Support_vector_machines Machine_learning',
 'Latent_variable_models Machine_learning',
 'Cluster_analysis Machine_learning',
 'Signal_processing_conferences Machine_learning',
 'Artificial_intelligence_conferences Machine_learning',
 'Data_mining_and_machine_learning_software Machine_learning',
 'Machine_learning_algorithms Machine_learning',
 'Bayesian_networks Machine_learning',
 'Machine_learning_portal Machine_learning',
 'Statistical_natural_language_processing Machine_learning',
 'Markov_models Machine_learning',
 'Evolutionary_algorithms Machine_learning',
 'Genetic_programming Machine_learning',
 'Datasets_in_machine_learning Machine_learning',
 'Dimension_reduction Machine_learning',
 'Machine_learning_researchers Machine_learning',
 'Supervised_learning Machine_

In [29]:
#save graph to files
with open('nodes.txt', 'w') as f:
    for item in all_nodes:
        f.write("%s\n" % item)

with open('relations.txt', 'w') as f:
    for item in all_relations:
        f.write("%s\n" % item)        